In [7]:
import pandas as pd
import requests
import time

# Load your CSV data
df = pd.read_csv('Bus Reserved Lanes.csv')

# Create a set of unique intersections to geocode
intersections = set(df['From'].tolist() + df['To'].tolist())

# Dictionary to store geocoding results
geocoded = {}

# Example using Nominatim (add "Calgary, Alberta, Canada" to improve accuracy)
for intersection in intersections:
    query = f"{intersection}, Calgary, Alberta, Canada"
    url = f"https://nominatim.openstreetmap.org/search?q={query}&format=json&limit=1"
    
    # Add user-agent header as required by Nominatim
    headers = {'User-Agent': 'YourAppName/1.0'}
    
    response = requests.get(url, headers=headers)
    results = response.json()
    
    if results:
        lat = float(results[0]['lat'])
        lon = float(results[0]['lon'])
        geocoded[intersection] = (lat, lon)
    
    # Be nice to the API - rate limiting
    time.sleep(1)

# Add new columns to the dataframe
df['From_Lat'] = df['From'].map(lambda x: geocoded.get(x, (None, None))[0])
df['From_Lon'] = df['From'].map(lambda x: geocoded.get(x, (None, None))[1])
df['To_Lat'] = df['To'].map(lambda x: geocoded.get(x, (None, None))[0]) 
df['To_Lon'] = df['To'].map(lambda x: geocoded.get(x, (None, None))[1])

# Save the enriched dataset
df.to_csv('Bus_Reserved_Lanes_Geocoded-2.csv', index=False)

In [ ]:
import pandas as pd
import re

# Load your CSV data
df = pd.read_csv('/mnt/data/Bus Reserved Lanes.csv')

def extract_coordinates(google_earth_link):
    """Extracts latitude and longitude from a Google Earth URL."""
    match = re.search(r'@(.*?),(.*?),(.*?)[a-z]', google_earth_link)
    if match:
        lat, lon = float(match.group(1)), float(match.group(2))
        return lat, lon
    return None, None

# Sample Google Earth links (replace with actual mappings in practice)
links = {
    "9 Avenue Southeast & 15 Street Southeast": "https://earth.google.com/web/search/9+Avenue+Southeast+%26+15+Street+Southeast,+Calgary,+AB/@51.0377584,-114.0252312,1037.45499669a,507.83883874d,35y,0h,0t,0r/data=CiwiJgokCSS-Pm4chklAEbo9dqv8hElAGX6SY9b_gVzAIdhCsqZPg1zAQgIIATIpCicKJQohMThpTVgtWXdBcW9jYmdDVjFjdVhTVVFLQ3czWElPMUpUIAE6AwoBMEICCABKCAjHo4v-AhAB"
}

# Extract coordinates for "From" and "To" locations
df['From_Lat'], df['From_Lon'] = zip(*df['From'].map(lambda x: extract_coordinates(links.get(x, ""))))
df['To_Lat'], df['To_Lon'] = zip(*df['To'].map(lambda x: extract_coordinates(links.get(x, ""))))

# Save the updated dataset
df.to_csv('/mnt/data/Bus_Reserved_Lanes_Geocoded.csv', index=False)

print("Coordinates extracted and saved successfully.")

In [ ]:
# import pandas as pd
# import time
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager

# # Load your CSV data
# df = pd.read_csv('Bus Reserved Lanes.csv')

# def get_coordinates_google_earth(location):
#     """Fetches latitude and longitude by searching Google Earth."""
#     options = webdriver.ChromeOptions()
#     options.add_argument("--headless")  # Run in headless mode
#     driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
#     try:
#         search_url = f"https://earth.google.com/web/search/{location.replace(' ', '+')},+Calgary,+Alberta"
#         driver.get(search_url)
#         time.sleep(5)  # Wait for page to load
        
#         # Extract coordinates from URL
#         current_url = driver.current_url
#         coords = current_url.split("/@")[1].split(",")[:2]  # Extract lat, lon
        
#         return float(coords[0]), float(coords[1])
#     except Exception as e:
#         print(f"Error fetching {location}: {e}")
#         return None, None
#     finally:
#         driver.quit()

# # Extract coordinates for "From" and "To" locations
# df['From_Lat'], df['From_Lon'] = zip(*df['From'].map(lambda x: get_coordinates_google_earth(x)))
# df['To_Lat'], df['To_Lon'] = zip(*df['To'].map(lambda x: get_coordinates_google_earth(x)))

# # Save the updated dataset
# df.to_csv('/mnt/data/Bus_Reserved_Lanes_Geocoded.csv', index=False)

# print("Coordinates extracted and saved successfully.")


Error fetching Edmonton TR: list index out of range
Error fetching Centre ST and 43 AV: list index out of range
Error fetching Centre ST and 54 AV: list index out of range
Error fetching 9 AV and 8 ST: list index out of range
Error fetching 9 AV and 15 ST: list index out of range


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import folium
import random

# For this example, let's create a synthetic geocoded dataset
# In reality, you would use the actual geocoded CSV from the previous step

# Define some base coordinates for Calgary
calgary_center = [51.0447, -114.0719]  # Calgary downtown approximately

# Create synthetic geocoded data based on the original dataset
# Sample coordinates for different quadrants of Calgary
quadrant_coords = {
    'NE': {'base_lat': 51.0800, 'base_lon': -114.0100, 'variation': 0.015},
    'NW': {'base_lat': 51.0800, 'base_lon': -114.1300, 'variation': 0.015},
    'SE': {'base_lat': 51.0100, 'base_lon': -114.0100, 'variation': 0.015},
    'SW': {'base_lat': 51.0100, 'base_lon': -114.1300, 'variation': 0.015}
}

# Routes from the original CSV
routes = [
    {'id': 1, 'from': 'Ogden RD and Glenmore TR South Int', 'to': 'Ogden RD and Glenmore TR North Int', 'quadrant': 'SE', 'lane_type': 'Bus Only Lane', 'length': 185},
    {'id': 2, 'from': 'Bowness RD and 16 AV', 'to': 'Bowness RD and Shaganappi TR', 'quadrant': 'NW', 'lane_type': 'Part Time Reserved Bus Lane', 'length': 216},
    {'id': 4, 'from': 'Centre ST and 43 AV', 'to': 'Centre ST and Mcknight BV', 'quadrant': 'NE', 'lane_type': 'Bus Only Lane', 'length': 640},
    {'id': 6, 'from': '9 AV and 8 ST', 'to': '9 AV and 15 ST', 'quadrant': 'SE', 'lane_type': 'Part Time Reserved Bus Lane', 'length': 1299},
    {'id': 8, 'from': 'Crowchild TR and 33 AV', 'to': 'Crowchild TR and Bow TR', 'quadrant': 'SW', 'lane_type': 'Bus Shoulder Lane', 'length': 1645},
    {'id': 10, 'from': 'Glenmore TR and Crowchild TR', 'to': 'Glenmore TR and 14 ST', 'quadrant': 'SW', 'lane_type': 'Bus Shoulder Lane', 'length': 1468},
    {'id': 15, 'from': '17 AV SE and 28 ST SE', 'to': '9 AV SE and 16 ST SE', 'quadrant': 'SE', 'lane_type': 'Max Purple BRT Dedicated Transitway', 'length': 2125},
    {'id': 17, 'from': 'Glenmore TR SW and 14 ST SW', 'to': 'Southland DR SW and Bradbury DR SW', 'quadrant': 'SW', 'lane_type': 'Max Yellow SW BRT Dedicated Transitway', 'length': 4019}
]

# Generate synthetic coordinates for each route
data = []
for route in routes:
    quadrant = route['quadrant']
    if quadrant not in quadrant_coords:
        quadrant = 'NE'  # Default to NE if quadrant not found
    
    # Generate from coordinates
    from_lat = quadrant_coords[quadrant]['base_lat'] + random.uniform(-0.015, 0.015)
    from_lon = quadrant_coords[quadrant]['base_lon'] + random.uniform(-0.015, 0.015)
    
    # Generate to coordinates - make them nearby but different
    # Adjust the to_point to create a sensible line length relative to the 'length' attribute
    length_factor = route['length'] / 1000.0  # Convert meters to rough coordinate difference
    
    # Ensure the direction of the line makes sense for the quadrant
    dir_factor = 1 if random.random() > 0.5 else -1
    to_lat = from_lat + (dir_factor * length_factor * random.uniform(0.002, 0.005))
    to_lon = from_lon + (dir_factor * length_factor * random.uniform(0.002, 0.005))
    
    data.append({
        'ID Number': route['id'],
        'From': route['from'],
        'To': route['to'],
        'Quadrant': quadrant,
        'Lane Type': route['lane_type'],
        'Length m': route['length'],
        'From_Lat': from_lat,
        'From_Lon': from_lon,
        'To_Lat': to_lat,
        'To_Lon': to_lon
    })

# Create a DataFrame
df = pd.DataFrame(data)

# Create the map centered on Calgary
m = folium.Map(location=calgary_center, zoom_start=12)

# Add a basemap
folium.TileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 
                 attr='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors').add_to(m)

# Define colors for different types of bus lanes
lane_colors = {
    'Bus Only Lane': 'red',
    'Part Time Reserved Bus Lane': 'blue',
    'Bus Shoulder Lane': 'green',
    'Max Purple BRT Dedicated Transitway': 'purple',
    'Max Yellow SW BRT Dedicated Transitway': 'orange'
}

# Add bus lanes to the map
for _, row in df.iterrows():
    # Create a line for each bus lane
    points = [
        [row['From_Lat'], row['From_Lon']],
        [row['To_Lat'], row['To_Lon']]
    ]
    
    # Get color based on lane type
    lane_type = row['Lane Type']
    color = lane_colors.get(lane_type, 'gray')
    
    # Add the line
    folium.PolyLine(
        points,
        color=color,
        weight=5,
        opacity=0.7,
        tooltip=f"ID: {row['ID Number']}<br>Type: {lane_type}<br>Length: {row['Length m']}m"
    ).add_to(m)
    
    # Add markers for the start and end points
    folium.CircleMarker(
        location=[row['From_Lat'], row['From_Lon']],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=1.0,
        tooltip=f"From: {row['From']}"
    ).add_to(m)
    
    folium.CircleMarker(
        location=[row['To_Lat'], row['To_Lon']],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=1.0,
        tooltip=f"To: {row['To']}"
    ).add_to(m)

# Add a legend
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; right: 50px; 
            border:2px solid grey; z-index:9999; 
            background-color:white;
            padding: 10px;
            border-radius: 5px;
            ">
     <p><b>Bus Lane Types</b></p>
'''

for lane_type, color in lane_colors.items():
    legend_html += f'<p><span style="color:{color};"><b>■</b></span> {lane_type}</p>'

legend_html += '</div>'

m.get_root().html.add_child(folium.Element(legend_html))

# Save the map to an HTML file
m.save('calgary_bus_lanes_map.html')

print("Map saved to calgary_bus_lanes_map.html")

Enhanced CSV saved to enhanced_bus_lanes.csv
Map saved to calgary_bus_lanes_map.html

Enhanced CSV structure:
   ID Number                                From  \
0          1  Ogden RD and Glenmore TR South Int   
1          2                Bowness RD and 16 AV   
2          4                 Centre ST and 43 AV   
3          6                       9 AV and 8 ST   
4          8              Crowchild TR and 33 AV   

                                   To                    Lane Type  
0  Ogden RD and Glenmore TR North Int                Bus Only Lane  
1        Bowness RD and Shaganappi TR  Part Time Reserved Bus Lane  
2           Centre ST and Mcknight BV                Bus Only Lane  
3                      9 AV and 15 ST  Part Time Reserved Bus Lane  
4             Crowchild TR and Bow TR            Bus Shoulder Lane  

Geometry column contains multiline string representing path coordinates


In [3]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import os

def plot_calgary_bus_lanes(csv_file_path):
    """
    Read CSV file containing Calgary bus lane data and create an interactive map visualization.
    
    Parameters:
    csv_file_path (str): Path to the CSV file containing bus lane data
    
    Returns:
    folium.Map: Interactive map object with bus lanes plotted
    """
    # Read the CSV file
    df = pd.read_csv(csv_file_path)
    
    # Filter out rows with invalid coordinates
    df_valid = df[(df['From_Lat'] != 'Invalid') & 
                  (df['From_Lon'] != 'Invalid') &
                  (df['To_Lat'] != 'Invalid') & 
                  (df['To_Lon'] != 'Invalid') &
                  (df['From_Lat'] != 'unknown') & 
                  (df['From_Lon'] != 'unknown') &
                  (df['To_Lat'] != 'unknown') & 
                  (df['To_Lon'] != 'unknown')]
    
    # Convert coordinates to float
    for col in ['From_Lat', 'From_Lon', 'To_Lat', 'To_Lon']:
        df_valid[col] = pd.to_numeric(df_valid[col], errors='coerce')
    
    # Drop rows with NaN values in coordinates
    df_valid = df_valid.dropna(subset=['From_Lat', 'From_Lon', 'To_Lat', 'To_Lon'])
    
    # Initialize the map centered on Calgary
    calgary_map = folium.Map(location=[51.0447, -114.0719], zoom_start=12, 
                             tiles='OpenStreetMap')
    
    # Create a marker cluster for single points (where from and to are the same)
    marker_cluster = MarkerCluster().add_to(calgary_map)
    
    # Define color mapping for different bus lane types
    def get_lane_color(lane_type):
        if 'Part Time' in lane_type:
            return '#3498db'  # Blue for part-time bus lanes
        elif 'Left Turn' in lane_type:
            return '#2ecc71'  # Green for left turn lanes
        elif 'Ramp' in lane_type:
            return '#f39c12'  # Orange for ramps
        else:
            return '#e74c3c'  # Red for bus only lanes
    
    # Define line weight based on length
    def get_line_weight(length):
        try:
            length = float(length)
            return 5 if length > 500 else 3
        except (ValueError, TypeError):
            return 3
    
    # Plot each bus lane
    for _, lane in df_valid.iterrows():
        # Create a popup with information about the bus lane
        popup_html = f"""
        <strong>Bus Lane ID: {lane['ID Number']}</strong><br>
        From: {lane['From']}<br>
        To: {lane['To']}<br>
        Direction: {lane['Direction']}<br>
        Type: {lane['Lane Type']}<br>
        Commission Date: {lane['Comission Date'] if pd.notna(lane['Comission Date']) else 'Unknown'}<br>
        Length: {lane['Length m'] if pd.notna(lane['Length m']) else 'Unknown'} m
        """
        
        # Check if from and to coordinates are the same
        if (lane['From_Lat'] == lane['To_Lat'] and lane['From_Lon'] == lane['To_Lon']):
            # Add a circle marker for single points
            folium.CircleMarker(
                location=[lane['From_Lat'], lane['From_Lon']],
                radius=8,
                popup=folium.Popup(popup_html, max_width=300),
                color='black',
                weight=1,
                fill_color=get_lane_color(lane['Lane Type']),
                fill_opacity=0.8
            ).add_to(marker_cluster)
        else:
            # Add polyline for the bus lane
            folium.PolyLine(
                locations=[[lane['From_Lat'], lane['From_Lon']], 
                           [lane['To_Lat'], lane['To_Lon']]],
                popup=folium.Popup(popup_html, max_width=300),
                color=get_lane_color(lane['Lane Type']),
                weight=get_line_weight(lane['Length m']),
                opacity=0.7
            ).add_to(calgary_map)
    
    # Add legend to the map
    legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; right: 50px; 
                border:2px solid grey; z-index:9999; 
                background-color:white;
                padding: 10px;
                border-radius: 5px;
                ">
      <h4>Bus Lane Types</h4>
      <div>
        <span style="display:inline-block; width:15px; height:15px; 
                     background-color:#e74c3c; margin-right:5px;"></span>
        Bus Only Lane
      </div>
      <div>
        <span style="display:inline-block; width:15px; height:15px; 
                     background-color:#3498db; margin-right:5px;"></span>
        Part Time Reserved Bus Lane
      </div>
      <div>
        <span style="display:inline-block; width:15px; height:15px; 
                     background-color:#2ecc71; margin-right:5px;"></span>
        Bus Only Left Turn Lane
      </div>
      <div>
        <span style="display:inline-block; width:15px; height:15px; 
                     background-color:#f39c12; margin-right:5px;"></span>
        Bus Only Ramp
      </div>
    </div>
    '''
    calgary_map.get_root().html.add_child(folium.Element(legend_html))
    
    return calgary_map

if __name__ == "__main__":
    # Specify the path to your CSV file here
    csv_path = "Manual-BusLane-Processed.csv"
    
    # Create the map
    bus_lanes_map = plot_calgary_bus_lanes(csv_path)
    
    # Save the map to an HTML file
    output_file = "calgary_bus_lanes_map.html"
    bus_lanes_map.save(output_file)
    
    print(f"Map has been saved to {os.path.abspath(output_file)}")

Map has been saved to c:\Users\bilal\Desktop\Bilal_Civil_Masters\02_Winter-2025\ENCI_619-4\visualization and analysis\Data Preprocessing\calgary_bus_lanes_map.html


C:\Users\bilal\AppData\Local\Temp\ipykernel_52628\915241389.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid[col] = pd.to_numeric(df_valid[col], errors='coerce')


c:\Users\bilal\anaconda3\envs\endg511\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


FileNotFoundError: [Errno 2] No such file or directory: 'zTransit_route_analysis.csv'